# Generating refactored corpus
Modify the synthetic benchmarks to generate a corpus of refactored programs.
Generate 1 refactored program for each benchmark version.

In [1]:
%load_ext autoreload
%autoreload 2
import refactorings
import projectslib
from pathlib import Path
import difflib
import random
import shutil
import subprocess
import random

## Look for ground truth
Ground truth means:
1. Buggy and OK versions both have some flaw location matching with the meta-alerts in the DB
2. Of the meta-alerts matched to flaw locations, buggy and OK should share conditions for at least 1 meta-alert.

In [40]:
corebench_project_names = ['make', 'grep', 'coreutils', 'findutils']
project_filter = []
project_filter += ['abm']
project_filter += ['ctestsuite']
project_filter += ['zitser']
project_filter += ['toyota']
# project_filter += corebench_project_names
projects = projectslib.get_projects(project_filter)
len(projects)

127

## Begin experiment

Subject benchmarks:
- ABM
- C Test Suite

Hyperparameters:
- random seed
- num_iterations: Number of transformations to do
- transforms:


In [36]:
from contextlib import redirect_stdout
from tqdm import tqdm_notebook
from pathlib import Path
import pandas as pd
import re
from shutil import ignore_patterns
import os

# Experiment parameters
picker = refactorings.random_picker
transforms = refactorings.all_refactorings
factory = refactorings.TransformationsFactory(transforms, picker)

# Zonk existing log files
logfile = Path('nb/log.txt')
if logfile.exists():
    logfile.unlink()
errors_log = Path('errors.log')
if errors_log.exists():
    errors_log.unlink()


def get_exclude(bench_name, project, c_file):
    """Return the exclude pattern given a benchmark name, project directory and target C file."""
    srcfiles = ['*.c.diff', '*.c.back', '*.c.refactor', '*.c.transforms']
    if bench_name == 'toyota':
        for f in project.glob('src/*.c'):
            if f.name == 'main.c' or f.name == c_file.name:
                continue
            else:
                srcfiles.append(f.name)
    return ignore_patterns(*srcfiles)


def proj_and_files(proj):
    result = set()
    for f, _ in proj.flaws:
        result.add((proj, f))
    return list(result)


print(f'Redirecting stdout to {logfile}')
for project_name in sorted(list(set(p.program for p in projects))):
    projects_and_files = []
    for p in projects:
        if p.program == project_name:
            result = set()
            for f, _ in p.flaws:
                result.add((p, f))
            projects_and_files.extend(result)
    projects_and_files = sorted(projects_and_files, key=lambda p: str(p[1]))
    for proj, c_file in tqdm_notebook(projects_and_files, desc=project_name):
        avoid = []
        for f, lineno in proj.flaws:
            if f == c_file:
                avoid.append(lineno)
        assert len(avoid) > 0
        random.seed(0)
        with open(logfile, 'a') as f:
            with redirect_stdout(f):
                exclude = get_exclude(proj.program, proj.buggy_path, c_file)
                with factory.make_project(c_file, proj.buggy_path, exclude, avoid=avoid) as refactoring_project:
                    print('***REFACTORING***', refactoring_project, c_file)
                    tmp_c_file, transforms_applied = refactoring_project.apply_all(return_applied=True)
                    new_c_file = Path(str(c_file) + '.refactor')
                    shutil.copy(tmp_c_file, new_c_file)
                    with open(c_file.parent / (c_file.name + '.transforms'), 'w') as f:
                        f.write('\n'.join(t.__name__ for t in transforms_applied))
                    with open(c_file) as old_f, open(new_c_file) as new_f:
                        diff = list(difflib.unified_diff(old_f.readlines(), new_f.readlines(), fromfile=c_file.name, tofile=c_file.name))
                    with open(str(c_file) + '.diff', 'w') as f:
                        f.write(''.join(diff))

Redirecting stdout to log.txt


In [41]:
import pandas as pd
import difflib

all_transform_names = sorted([t.__name__ for t in refactorings.all_refactorings])

with open('nb/stats.csv', 'w') as csv_f:
    print(','.join(('project', 'version', *all_transform_names, 'lines changed')), file=csv_f)
    for proj in projects:
        # Get files and check they exist
        try:
            transforms_file = next(proj.buggy_path.walk('*.c.transforms'))
            new_file = transforms_file.with_suffix('.refactor')
            old_file = new_file.parent / new_file.stem
        except:
            raise Exception(f'{proj.program}-{proj.buggy}')
        assert transforms_file.exists()
        assert new_file.exists(), new_file
        assert old_file.exists(), old_file

        # Collect which transforms were applied
        with open(transforms_file) as f:
            transforms_applied = set(f.read().splitlines())
        was_applied = [t in transforms_applied for t in all_transform_names]

        # Collect number of changed lines
        differences = sum(1 for d in difflib.ndiff(old_file.open().readlines(), new_file.open().readlines()) if d[0] in ('+', '-'))

        print(','.join((proj.program, proj.buggy, *('TRUE' if a else 'FALSE' for a in was_applied), str(differences))), file=csv_f)

In [39]:
import subprocess
import pandas as pd
from pathlib import Path
import tqdm.notebook as tqdm
import shutil
import re

exclude = []
failed = []
pbar = tqdm.tqdm(projects)
for proj in pbar:
    project = proj.program
    code_dir = proj.buggy_path
    assert code_dir.exists()

    # Replace refactored file
    proj_files = {f for f, _ in proj.flaws}
    proj_files = list(sorted(proj_files))
    for original in proj_files:
        pbar.set_postfix({'excluded': len(exclude), 'failed': len(failed), 'file': original})
        refactored = original.with_suffix('.c.refactor')
        backup = refactored.with_suffix('.back')
        assert refactored.exists(), refactored
        assert original.exists(), original
        if not backup.exists():
            shutil.copy2(original, backup)
        assert backup.exists(), backup
        
        # Fix up the makefile which might reference the *-ok/*-bad filenames
        makefile = refactored.parent / 'Makefile'
        assert makefile.exists()
        makefile_backup = refactored.parent / 'Makefile.okbad'
        if makefile_backup.exists():
            shutil.copy2(makefile_backup, makefile)
        else:
            shutil.copy2(makefile, makefile_backup)
        with open(makefile) as f:
            text = f.read()
        text = re.sub(r'(\w+)-ok', r'\1', text)
        text = re.sub(r'(\w+)-bad', r'\1', text)
        if (proj.program, proj.buggy) in (('ctestsuite', '095'), ('ctestsuite', '097'), ('ctestsuite', '099'), ('ctestsuite', '187')):
            text = text.replace('$(CC) $(CFLAGS) $(SRC)', '$(CC) $(CFLAGS) $(SRC) $(shell mysql_config --cflags --libs)')
        if (proj.program, proj.buggy) in (('zitser', '287'), ('zitser', '289'), ('zitser', '295')):
            text = text.replace('create\n', './create\n')
        with open(makefile, 'w') as f:
            f.write(text)

        # Build and check
        proc = subprocess.run('make clean', cwd=str(code_dir), capture_output=True, shell=True)
        proc = subprocess.run('make', cwd=str(code_dir), capture_output=True, shell=True)
        if proc.returncode != 0:
            exclude.append((project, proj.buggy, proc.stderr.decode()))
            continue
        
        try:
            shutil.copy(refactored, original)
            proc = subprocess.run('make clean', cwd=str(code_dir), capture_output=True, shell=True)
            proc = subprocess.run('make', cwd=str(code_dir), capture_output=True, shell=True)
            if proc.returncode != 0:
                failed.append((project, proj.buggy, proc.stderr.decode()))
        finally:
            shutil.copy(backup, original)

# install mysql-devel and pam-devel first.
# Some projects in ctestsuite won't build.
print(len(exclude), 'versions didn\'t build')
with open('nb/exclude.csv', 'w') as f:
    f.write(f'project,version\n')
    for project, version, e in exclude:
        f.write(f'{project},{version}\n')
with open('nb/exclude.log', 'w') as f:
    for project, version, e in exclude:
        f.write(f'***tests/{project}/{version}***\n')
        f.write(f'{e}\n')

print(len(failed), 'versions failed after refactoring')
with open('nb/failed.csv', 'w') as f:
    f.write(f'project,version\n')
    for project, version, e in failed:
        f.write(f'{project},{version}\n')
with open('nb/failed.log', 'w') as f:
    for project, version, e in failed:
        f.write(f'***tests/{project}/{version}***\n')
        f.write(f'{e}\n')


0 versions didn't build
1 versions failed after refactoring


To pack up refactorings, use `tar cf refactors.tar $(find itc -name '*.c.reformat' -o -name '*.c.diff' -o -name '*.transforms.txt')`.